In [22]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score, roc_curve, auc, f1_score
from sklearn.model_selection import train_test_split

from components import data_handling, glrp_scipy,nn_cnn_models
from lib import models, graph, coarsening

# from sklearn.model_selection import StratifiedKFold

import time


In [3]:

rndm_state = 7
np.random.seed(rndm_state)

In [4]:

path_to_feature_val = "./data/GE_PPI/GEO_HG_PPI.csv"
path_to_feature_graph = "./data/GE_PPI/HPRD_PPI.csv"
path_to_labels = "./data/GE_PPI/labels_GEO_HG.csv"

DP = data_handling.DataPreprocessor(path_to_feature_values=path_to_feature_val, path_to_feature_graph=path_to_feature_graph,
                                    path_to_labels=path_to_labels)
X = DP.get_feature_values_as_np_array()  # gene expression
A = csr_matrix(DP.get_adj_feature_graph_as_np_array().astype(np.float32))  # adjacency matrix of the PPI network
y = DP.get_labels_as_np_array()  # labels

print("GE data, X shape: ", X.shape)
print("Labels, y shape: ", y.shape)
print("PPI network adjacency matrix, A shape: ", A.shape)

X_train_unnorm, X_test_unnorm, y_train, y_test = train_test_split(X, y, test_size=0.10,
                                                                    stratify=y, random_state=rndm_state)

# Need to know which patients got into train and test subsets
_, _, patient_indexes_train, patient_indexes_test = train_test_split(X, DP.labels.columns.values.tolist(), test_size=0.10,
                                                                    stratify=y, random_state=rndm_state)

# Data frame with test patients and corresponding ground truth labels
patient_ind_test_df = pd.DataFrame(data={"Patient ID": patient_indexes_test, "label": y_test})

# !!!
# Making data lying in the interval [0, 8.35]
X_train = X_train_unnorm - np.min(X)
X_test = X_test_unnorm - np.min(X)

print("X_train max", np.max(X_train))
print("X_train min", np.min(X_train))
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train, shape: ", y_train.shape)
print("y_test, shape: ", y_test.shape)

# coarsening the PPI graph to perform pooling in the model
graphs, perm = coarsening.coarsen(A, levels=2, self_connections=False)
L = [graph.laplacian(A, normalized=True) for A in graphs]

X_train = coarsening.perm_data(X_train, perm)
X_test = coarsening.perm_data(X_test, perm)

n_train = X_train.shape[0]

params = dict()
params['dir_name']       = 'GE'
params['num_epochs']     = 100
params['batch_size']     = 109
params['eval_frequency'] = 40

# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b1relu'
params['pool']           = 'mpool1'

# Number of classes.
C = y.max() + 1
assert C == np.unique(y).size

# Architecture.
params['F']              = [32, 32]  # Number of graph convolutional filters.
params['K']              = [8, 8]  # Polynomial orders.
params['p']              = [2, 2]    # Pooling sizes.
params['M']              = [512, 128, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 1e-4
params['dropout']        = 1
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0
params['decay_steps']    = n_train / params['batch_size']

model = models.cgcnn(L, **params)

GE data, X shape:  (969, 6888)
Labels, y shape:  (969,)
PPI network adjacency matrix, A shape:  (6888, 6888)
X_train max 8.352902354001749
X_train min 0.0
X_train shape:  (872, 6888)
X_test shape:  (97, 6888)
y_train, shape:  (872,)
y_test, shape:  (97,)
Layer 0: M_0 = |V| = 10032 nodes (3144 added),|E| = 27841 edges
Layer 1: M_1 = |V| = 5016 nodes (755 added),|E| = 24141 edges
Layer 2: M_2 = |V| = 2508 nodes (0 added),|E| = 20997 edges
NN architecture
  input: M_0 = 10032
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 10032 * 32 / 2 = 160512
    weights: F_0 * F_1 * K_1 = 1 * 32 * 8 = 256
    biases: F_1 = 32
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 5016 * 32 / 2 = 80256
    weights: F_1 * F_2 * K_2 = 32 * 32 * 8 = 8192
    biases: F_2 = 32
  layer 3: fc1
    representation: M_3 = 512
    weights: M_2 * M_3 = 80256 * 512 = 41091072
    biases: M_3 = 512
  layer 4: fc2
    representation: M_4 = 128
    weights: M_3 * M_4 = 512 * 128 = 65536
    biases: M_4 = 1

Since CGCNN cant work--> need to have another model test

In [24]:
# Additional parameters needed
params["L"] = L

model = nn_cnn_models.get_cheb_net_model(**params)
my_cheb_net_for_cv = nn_cnn_models.MyChebNet(params, model)
my_cheb_net_for_cv.create(feature_number=None)

# model.summary()

start = time.time()
my_cheb_net_for_cv.fit(x=np.expand_dims(X_train, 2), y=y_train, validation_data=[np.expand_dims(X_test, 2), y_test],
                        verbose=1)
end = time.time()
print("\n\tTraining time:", end-start, "\n")
# y_preds = my_cheb_net_for_cv.predict(X_test)

y_preds = my_cheb_net_for_cv.predict(np.expand_dims(X_test, 2))
acc = accuracy_score(y_test, np.argmax(y_preds, axis=1))
f1 = f1_score(y_test, np.argmax(y_preds, axis=1), average='weighted')
print("test Accuraccy: %0.4f, test F1: %0.4f" % (acc, f1))


	Cheb_conv, first layer, input shape : 10032
	mpool1, Pooling layer, size : 2
	Cheb_conv layer, input shape : 5016
	mpool1, Pooling layer, size : 2
	FC layer, nodes: 512
	FC layer, nodes: 128
	Last layer, nodes: 2 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cheb_conv (ChebConv)        (None, 10032, 32)         288       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 5016, 32)         0         
 )                                                               
                                                                 
 cheb_conv_1 (ChebConv)      (None, 5016, 32)          8224      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2508, 32)         0         
 1D)                                                             
                                     

c:\Users\chant\miniconda3\envs\lrpenv\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


872/872 [==============================] - 6s 7ms/sample - loss: 42.9655 - acc: 0.4931 - val_loss: 13.1091 - val_acc: 0.5979
Epoch 2/100
872/872 [==============================] - 5s 5ms/sample - loss: 5.7718 - acc: 0.5367 - val_loss: 1.7154 - val_acc: 0.4021
Epoch 3/100
872/872 [==============================] - 5s 5ms/sample - loss: 0.9681 - acc: 0.5688 - val_loss: 0.7171 - val_acc: 0.6701
Epoch 4/100
872/872 [==============================] - 5s 5ms/sample - loss: 0.7652 - acc: 0.6365 - val_loss: 0.7610 - val_acc: 0.6186
Epoch 5/100
872/872 [==============================] - 5s 5ms/sample - loss: 0.6921 - acc: 0.6858 - val_loss: 0.6600 - val_acc: 0.7526
Epoch 6/100
872/872 [==============================] - 5s 5ms/sample - loss: 0.6522 - acc: 0.7225 - val_loss: 0.6396 - val_acc: 0.7835
Epoch 7/100
872/872 [==============================] - 5s 5ms/sample - loss: 0.6253 - acc: 0.7477 - val_loss: 0.6374 - val_acc: 0.7526
Epoch 8/100
872/872 [==============================] - 5s 5ms/sam

c:\Users\chant\miniconda3\envs\lrpenv\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


test Accuraccy: 0.7938, test F1: 0.7952


In [31]:
y_preds = my_cheb_net_for_cv.predict(np.expand_dims(X_test, 2))
acc = accuracy_score(y_test, np.argmax(y_preds, axis=1))
f1 = f1_score(y_test, np.argmax(y_preds, axis=1), average='weighted')
print("\tTest F1 weighted: ", f1)
print("\tTest Accuraccy:", acc, "\n")

	Test F1 weighted:  0.7951614641330508
	Test Accuraccy: 0.7938144329896907 



In [30]:
probas_ = model.predict(np.expand_dims(X_test, 2))
labels_by_network = np.argmax(probas_, axis=1)

fpr, tpr, _ = roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)
print("\n\tTest AUC:", roc_auc) # np.argmax(y_preds, axis=2)[:, 0] fot categorical


	Test AUC: 0.8470380194518126


skip this this is for CGCNN

In [25]:
probas_ = model.get_probabilities(X_test)
labels_by_network = np.argmax(probas_, axis=1)

fpr, tpr, _ = roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)
f1 = 100 * f1_score(y_test, labels_by_network, average='weighted')
acc = accuracy_score(y_test, labels_by_network)
print("\n\tTest AUC:", roc_auc) # np.argmax(y_preds, axis=2)[:, 0] fot categorical
print("\tTest F1 weighted: ", f1)
print("\tTest Accuraccy:", acc, "\n")




AttributeError: 'Sequential' object has no attribute 'get_probabilities'

In [32]:
# !!!
# Creating hot-encoded labels for GLRP
I = np.eye(C)
tmp = I[labels_by_network]
labels_hot_encoded = np.ones((model.batch_size, C))
labels_hot_encoded[0:tmp.shape[0], 0:tmp.shape[1]] = tmp
print("labels_hot_encoded.shape", labels_hot_encoded.shape)

dir_to_save = "./results/"

print("labels_by_network type", labels_by_network.dtype)
print("y_test type", y_test.dtype)
concordance = y_test == labels_by_network
print(y_test)
print(labels_by_network)
print(concordance)
concordance = concordance.astype(int)
out_labels_conc_df = pd.DataFrame(np.array([labels_by_network, concordance]).transpose(),
                                    columns=["Predicted", "Concordance"])
concordance_df = patient_ind_test_df.join(out_labels_conc_df)
concordance_df.to_csv(path_or_buf = dir_to_save + "predicted_concordance_lrp0.csv", index=False)



AttributeError: 'Sequential' object has no attribute 'batch_size'

In [8]:
    # !!!
    # CALCULATION OF RELEVANCES
    # CAN TAKE QUITE SOME TIME (UP to 10 MIN, Intel(R) Xeon(R) CPU E5-1620 v2 @ 3.70GHz, 32 GB RAM)
from components import glrp_0_scipy
glrp = glrp_scipy.GraphLayerwiseRelevancePropagation(model, X_test, labels_hot_encoded)
rel = glrp.get_relevances()[-1][:X_test.shape[0], :]
rel = coarsening.perm_data_back(rel, perm, X.shape[1])
rel_df = pd.DataFrame(rel, columns=DP.feature_names)
rel_df = pd.DataFrame(data={"Patient ID": patient_indexes_test}).join(rel_df)
rel_df.to_csv(path_or_buf = dir_to_save + "relevances_rendered_class_lrp.csv", index=False)



	Calculating Polynomials of Laplace Matrices... Time:  18.933253288269043 

INFO:tensorflow:Restoring parameters from C:\Users\chant\OneDrive\Desktop\XAI\graph-lrp\lib\..\checkpoints\GE\model-800

    Relevance calculation:
	Fully connected: logits
	Fully connected: fc2
	Fully connected: fc1
	Flatten layer: flatten
	Pooling: conv2 pooling
		name of pooling: mpool1
	Convolution:  conv2 

INFO:tensorflow:Restoring parameters from C:\Users\chant\OneDrive\Desktop\XAI\graph-lrp\lib\..\checkpoints\GE\model-800
INFO:tensorflow:Restoring parameters from C:\Users\chant\OneDrive\Desktop\XAI\graph-lrp\lib\..\checkpoints\GE\model-800
INFO:tensorflow:Restoring parameters from C:\Users\chant\OneDrive\Desktop\XAI\graph-lrp\lib\..\checkpoints\GE\model-800

	conv2, relevance propagation time is:  341.6468632221222
	Pooling: conv1 pooling
		name of pooling: mpool1
	Convolution, the first layer: conv1 

INFO:tensorflow:Restoring parameters from C:\Users\chant\OneDrive\Desktop\XAI\graph-lrp\lib\..\checkp

In [9]:
import quantus

c:\Users\chant\miniconda3\envs\lrpenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
explanations = rel_df.to_numpy()
print(X_test.shape)
quantus.FaithfulnessCorrelation().evaluate_instance(model,X_test,y_test,explanations,labels_by_network)

(97, 10032)
Warnings and information:
 (1) The Faithfulness Correlation metric is likely to be sensitive to the choice of baseline value 'perturb_baseline', size of subset |S| 'subset_size' and the number of runs (for each input and explanation pair) 'nr_runs'.  
 (2) If attributions are normalised or their absolute values are taken it may destroy or skew information in the explanation and as a result, affect the overall evaluation outcome.
 (3) Make sure to validate the choices for hyperparameters of the metric (by calling .get_params of the metric instance).
 (4) For further information, see original publication: Bhatt, Umang, Adrian Weller, and José MF Moura. 'Evaluating and aggregating feature-based model explanations.' arXiv preprint arXiv:2005.00631 (2020).
 (5) To disable these warnings set 'disable_warnings' = True when initialising the metric.



AttributeError: 'cgcnn' object has no attribute 'shape_input'